In [1]:
!pip install langchain langchain-community python-dotenv

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY') # For LLM -- llama-3.1-8b (small) & mixtral-8x7b-32768 (large)
os.environ['COHERE_API_KEY'] = os.getenv('COHERE_API_KEY') # For embedding

### Create Vectorstore

In [3]:
!pip install -U langchain-cohere

In [4]:
# !pip install tiktoken
# !pip install chromadb

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereEmbeddings

embedding_model = CohereEmbeddings(model="embed-english-v3.0")

# Docs to index
urls = [
    "https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-2-reflection/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 500,
    chunk_overlap = 0
)

doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorstore
vectorstore= Chroma.from_documents(
    documents= doc_splits,
    collection_name = "rag",
    embedding = embedding_model
)

retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":4}# number of documents to retrieve
)

In [6]:
question = "what are the differnt kind of agentic design patterns?"

In [7]:
docs = retriever.invoke(question)

### Check what our doc looklike

In [10]:
print(f"Title: {docs[1].metadata['title']}\n\nSource: {docs[1].metadata['source']}\n\nContent: {docs[1].page_content}")

Title: Agentic Design Patterns Part 5, Multi-Agent Collaboration

Source: https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io

Content: clear, efficient code as opposed to, say, scalable and highly secure code. By decomposing the overall task into subtasks, we can optimize the subtasks better.Perhaps most important, the multi-agent design pattern gives us, as developers, a framework for breaking down complex tasks into subtasks. When writing code to run on a single CPU, we often break our program up into different processes or threads. This is a useful abstraction that lets us decompose a task, like implementing a web browser, into subtasks that are easier to code. I find thinking through multi-agent roles to be a useful abstraction as well.In many companies, managers routinely decide what roles to hire, and then how to split complex projects — like writing a large piece of software or preparing a research repo

In [16]:
#!pip install langchain-groq
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from typing import Literal

# Data Model
class GradeDocuments(BaseModel):
    binary_score: Literal["yes", "no"] = Field(
        description="Return only 'yes' or 'no'."
    )

# LLM with function call
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

#prompt
system= """You are a grader assessing the relevance of a retrieved document.
Return ONLY a JSON object with a single key 'binary_score'.
The value must be either 'yes' or 'no'.
Do not add explanations or extra text."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}")
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

### Filter out the non-relevant docs

In [17]:
docs_to_use = []
for doc in docs:
  print(doc.page_content, "\n",'-'*50)
  res = retrieval_grader.invoke({"question":question , "document":doc.page_content})
  print(res,"\n")
  if res.binary_score == "yes":
    docs_to_use.append(doc)

GPT-3.5 to GPT-4 is dwarfed by incorporating an iterative agent workflow. Indeed, wrapped in an agent loop, GPT-3.5 achieves up to 95.1%. Open source agent tools and the academic literature on agents are proliferating, making this an exciting time but also a confusing one. To help put this work into perspective, I’d like to share a framework for categorizing design patterns for building agents. My team AI Fund is successfully using these patterns in many applications, and I hope you find them useful.Reflection: The LLM examines its own work to come up with ways to improve it. Tool Use: The LLM is given tools such as web search, code execution, or any other function to help it gather information, take action, or process data.Planning: The LLM comes up with, and executes, a multistep plan to achieve a goal (for example, writing an outline for an essay, then doing online research, then writing a draft, and so on).Multi-agent collaboration: More than one AI agent work together, splitting u

### Generate Result

In [18]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge.
Use three-to-five sentences maximum and keep the answer concise."""

prompt = ChatPromptTemplate.from_messages(
    {
        ("system", system),
        ("human","Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>")
    }
)

# LLM
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n".join(f"<doc{i+1}>:\nTitle:{doc.metadata['title']}\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i+1}>\n" for i, doc in enumerate(docs))

# Chain
rag_chain = prompt | llm | StrOutputParser()

generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question": question})
print(generation)

According to the retrieved documents, there are four key AI agentic design patterns: 

1. Reflection: The LLM examines its own work to come up with ways to improve it.
2. Tool Use: The LLM is given tools to help it gather information, take action, or process data.
3. Planning: The LLM comes up with and executes a multistep plan to achieve a goal.
4. Multi-agent collaboration: More than one AI agent work together to come up with better solutions than a single agent would.

These design patterns are further explained in the provided documents, with the last one being Multi-agent collaboration.


### Check for Hallucinations

In [19]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in 'generation' answer."""

    binary_score: str = Field(
        ...,
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

# LLM with function call
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n <facts>{documents}</facts> \n\n LLM generation: <generation>{generation}</generation>"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

response = hallucination_grader.invoke({"documents": format_docs(docs_to_use), "generation": generation})
print(response)

binary_score='yes'
